## Preprocessing

In [1]:
%pip install dask
%pip install distributed
%pip install AST
%pip install tqdm
import dask.dataframe as dd
from dask.distributed import Client

client = Client()
client

MINIMUM_REVIEWS = 100

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-bj5m43ew/ast_b30ffe33fef245758edcef33d03b587f/setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
        File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/codecs.py", line 906, in open
          file = builtins.open(filename, mode, buffering)
      FileNotFoundError: [Errno 2] No such file or directory: '/tmp/pip-install-bj5m43ew/ast_b30ffe33fef245758edcef33d03b587f/AST/README'
      [end of

In [2]:
yelp_restaurants = (
    dd.read_csv("./data/yelp-restaurants.csv/0.part", dtype={"postal_code": "object"})
    .compute()
    
)
yelp_restaurants = yelp_restaurants.drop(columns=yelp_restaurants.columns[0])

In [3]:
import ast
from tqdm import tqdm
import pandas as pd

def attribute_mapping(restaurants):
    restaurants = restaurants
    # map attributes to columns  
    rows = list(restaurants.index)

    attributes = pd.DataFrame()
    
    for row in tqdm(rows):
        d = restaurants.loc[row, "attributes"]
        attr = pd.DataFrame(eval(d), index=[row])
        # extra_layer_unfiltered = ['BusinessParking','GoodForMeal','Ambience']
        # extra_layer = [e for e in extra_layer_unfiltered if e in attr.columns]
        extra_layer = ["BusinessParking", "GoodForMeal", "Ambience"]

        for item in extra_layer:
            try:
                s = attr.loc[row, item]
            except KeyError:
                continue
            if type(s) == pd._libs.missing.NAType:
                continue
            col = pd.DataFrame(ast.literal_eval(s), index=[row])
            attr = pd.concat([attr, col], axis=1)

        attr.drop(columns=extra_layer, inplace=True, errors="ignore")
        attributes = pd.concat([attributes, attr], axis=0)

    # feature reduction

    # Missing Value Ratio: if more than 20% is null -> drop
    null_count = attributes.isna().sum()
    attributes.dropna(thresh=len(attributes) * 0.8, axis=1, inplace=True)

    # replace
    attributes.replace(
        to_replace={"False": "0", "True": "1", "None": pd.NA}, inplace=True
    )
    cols_to_process = ["Alcohol", "NoiseLevel", "RestaurantsAttire", "WiFi"]
    cols_to_int = [
        "BikeParking",
        "BusinessAcceptsCreditCards",
        "Caters",
        "GoodForKids",
        "HasTV",
        "OutdoorSeating",
        "RestaurantsDelivery",
        "RestaurantsGoodForGroups",
        "RestaurantsPriceRange2",
        "RestaurantsReservations",
        "RestaurantsTakeOut",
        "garage",
        "street",
        "validated",
        "lot",
        "valet",
        "romantic",
        "intimate",
        "touristy",
        "hipster",
        "divey",
        "classy",
        "trendy",
        "upscale",
        "casual",
    ]

    for col in cols_to_int:
        try:
            attributes[col] = attributes[col].astype(int, errors="ignore")
        except KeyError:
            continue
    attributes.Alcohol[attributes.Alcohol == "u'none'"] = 0
    attributes.Alcohol[attributes.Alcohol == "'none'"] = 0
    attributes.Alcohol[attributes.Alcohol == "u'beer_and_wine'"] = 1
    attributes.Alcohol[attributes.Alcohol == "'beer_and_wine'"] = 1
    attributes.Alcohol[attributes.Alcohol == "u'full_bar'"] = 2
    attributes.Alcohol[attributes.Alcohol == "'full_bar'"] = 2
    attributes.NoiseLevel[attributes.NoiseLevel == "u'quiet'"] = 0
    attributes.NoiseLevel[attributes.NoiseLevel == "'quiet'"] = 0
    attributes.NoiseLevel[attributes.NoiseLevel == "u'average'"] = 1
    attributes.NoiseLevel[attributes.NoiseLevel == "'average'"] = 1
    attributes.NoiseLevel[attributes.NoiseLevel == "u'loud'"] = 2
    attributes.NoiseLevel[attributes.NoiseLevel == "'loud'"] = 2
    attributes.NoiseLevel[attributes.NoiseLevel == "u'very_loud'"] = 3
    attributes.NoiseLevel[attributes.NoiseLevel == "'very_loud'"] = 3
    attributes.RestaurantsAttire[attributes.RestaurantsAttire == "u'casual'"] = 0
    attributes.RestaurantsAttire[attributes.RestaurantsAttire == "'casual'"] = 0
    attributes.RestaurantsAttire[attributes.RestaurantsAttire == "u'dressy'"] = 1
    attributes.RestaurantsAttire[attributes.RestaurantsAttire == "'dressy'"] = 1
    attributes.RestaurantsAttire[attributes.RestaurantsAttire == "u'formal'"] = 2
    attributes.RestaurantsAttire[attributes.RestaurantsAttire == "'formal'"] = 2
    attributes.WiFi[attributes.WiFi == "u'free'"] = "free"
    attributes.WiFi[attributes.WiFi == "u'no'"] = "no"
    attributes.WiFi[attributes.WiFi == "u'paid'"] = "paid"

    attributes.WiFi[attributes.WiFi == "'free'"] = "free"
    attributes.WiFi[attributes.WiFi == "'no'"] = "no"
    attributes.WiFi[attributes.WiFi == "'paid'"] = "paid"

    attributes = dd.from_pandas(attributes, npartitions=100)
    return attributes


In [4]:
def preprocess_yelp_restaurants(restaurants):
    # filter out non-restaurants
    restaurants = restaurants[
        restaurants["categories"].notna()
    ]  # filter out null attributes
    searchfor = ["food", "restaurant", "dessert"]
    restaurants = restaurants[
        restaurants["categories"].str.contains("|".join(searchfor), case=False)
    ]
    # filter out restaurants with less than MINIMUM_REVIEWS
    restaurants = restaurants[restaurants["review_count"] >= MINIMUM_REVIEWS]
    restaurants.reset_index(drop=True, inplace=True)

    # filter out null attributes
    restaurants = restaurants[restaurants["attributes"].notnull()]

    # add attributes
    attributes = attribute_mapping(restaurants).compute()
    restaurants = restaurants.merge(attributes, left_index=True, right_index=True)

    restaurants.fillna(restaurants.mode().iloc[0], inplace=True)

    # fix wifi
    wifi = dd.get_dummies(restaurants["WiFi"])
    restaurants = dd.concat([restaurants, wifi], axis=1)
    restaurants = restaurants.drop(columns="WiFi")
    attributes = attributes.drop(columns="WiFi")
    restaurants = restaurants.compute()

    return restaurants

In [5]:
# copy = yelp_restaurants.copy(deep=True)
# results = preprocess_yelp_restaurants(yelp_restaurants)
# results

In [6]:
yelp_reviews = dd.read_csv(
    "./data/yelp-reviews.csv/0.part", dtype={"postal_code": "object"}
).compute()

In [7]:
def preprocess_yelp_reviews(reviews):
    reviews = reviews.set_index("business_id")
    reviews = reviews[["stars", "useful", "funny", "cool"]]
    reviews = reviews.groupby("business_id").mean()
    reviews = reviews.rename(columns={"stars": "avg_stars"})
    reviews.compute()
    return reviews


# preprocess_yelp_reviews(yelp_reviews)

In [8]:
def preprocess_yelp(restaurants, reviews):
    df = restaurants.merge(
        reviews, on="business_id", how="left", suffixes=("", "_calculated")
    ).compute()
    return df

In [9]:
rest = preprocess_yelp_restaurants(yelp_restaurants)
rev = preprocess_yelp_reviews(yelp_reviews)
preprocessed = preprocess_yelp(rest,rev)

/tmp/ipykernel_152639/3454836335.py:8: PerformanceWarning: Falling back on a non-pyarrow code path which may decrease performance.
  restaurants["categories"].str.contains("|".join(searchfor), case=False)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

AttributeError: 'DataFrame' object has no attribute 'compute'

## Feature Selection

In [ ]:


features = (list(preprocessed.columns).remove("is_open"))
label = 'is_open'
features

df = restaurants[['stars','stars_calculated','review_count','RestaurantsPriceRange2',label]]
sns.set(rc = {'figure.figsize':(10,7)})
sns.heatmap(df.corr(),annot=True)

X = restaurants[features]
y = restaurants[label]
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

ros = RandomOverSampler(random_state=0)
rus = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X, y)

#feature selection

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

selector = SelectKBest(f_classif, k=5)
selector.fit(X, y)
filter = selector.get_support()
top_5_features = X.columns[filter]

print("Best 5 features:")
print(top_5_features)

# PCA
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples

zscoredData = stats.zscore(X)
pca = PCA().fit(zscoredData)
eigVals = pca.explained_variance_
loadings = pca.components_*-1
rotated = pca.fit_transform(zscoredData)*-1

#scree plot
numClasses = X.shape[1]
x = np.linspace(1,numClasses,numClasses)
plt.bar(x, eigVals, color='gray')
plt.plot([0,numClasses],[1,1],color='orange') # Orange Kaiser criterion line for the fox
plt.xlabel('Principal component')
plt.ylabel('Eigenvalue')
plt.show()

NameError: name 'preprocessed' is not defined

### Build and evaluate model

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1234)
#X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.30, random_state=1234)

In [ ]:
model = RandomForestClassifier()
model.fit(X_train,y_train)
pred = model.predict(X_test)
prob = model.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test,prob)

fig = plt.figure()
ax = fig.add_subplot(111)
sns.lineplot(x=fpr, y=tpr, marker = 'o')
plt.title("Receiver operating characteristic (ROC) curve")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
#plt.legend(['RF with 20 estimators', 'RF with 100 estimators'])
plt.show()

In [ ]:
auc(fpr, tpr)
accuracy_score(y_test, pred)

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve

pd.DataFrame(
confusion_matrix(y_test, pred, labels=[True, False]),
columns=['Predicted: Open', 'Predicted: Closed'],
index=['Actual: Open', 'Actual: Closed'])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

estimators = [("DT", DecisionTreeClassifier()),
              ("RF", RandomForestClassifier()),
              ("GBDT", GradientBoostingClassifier()),
              ("LR", LogisticRegression())
             ]
stacking_model = StackingClassifier(estimators=estimators,cv=5,passthrough=False)
params = {
    "DT__max_depth": [8, 16],
    "GBDT__n_estimators":[100,300],
    "RF__min_samples_split":[2,5,10]
}

stack_grid = GridSearchCV(stacking_model, params, cv=3, verbose=4, scoring='accuracy', refit=True, n_jobs=-1)
stack_grid.fit(X_train, y_train)
print(stack_grid.best_params_)

In [ ]:
best_estimators = [("DT", DecisionTreeClassifier(max_depth=stack_grid.best_params_['DT__max_depth'])),
              ("RF", RandomForestClassifier(min_samples_split=stack_grid.best_params_['RF__min_samples_split'])),
              ("GBDT", GradientBoostingClassifier(n_estimators=stack_grid.best_params_['GBDT__n_estimators'])),
              ("LR", LogisticRegression())
             ]

best_stacking_model = StackingClassifier(estimators=best_estimators,cv=5,passthrough=False)
best_stacking_model.fit(X_train,y_train)
ensemble_pred = best_stacking_model.predict(X_test)
ensemble_prob = best_stacking_model.predict_proba(X_test)[:,1]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,ensemble_prob)

fig = plt.figure()
ax = fig.add_subplot(111)
sns.lineplot(x=fpr, y=tpr, marker = 'o')
plt.title("Receiver operating characteristic (ROC) curve")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
#plt.legend(['RF with 20 estimators', 'RF with 100 estimators'])
plt.show()

In [ ]:
auc(fpr, tpr)
accuracy_score(y_test, ensemble_pred)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve

pd.DataFrame(
confusion_matrix(y_test, ensemble_pred, labels=[True, False]),
columns=['Predicted: Open', 'Predicted: Closed'],
index=['Actual: Open', 'Actual: Closed'])

precision_ensemble, recall_ensemble, thresholds_ensemble = precision_recall_curve(y_test,ensemble_prob)
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(y_test,prob)

fig = plt.figure()
ax = fig.add_subplot(111)
sns.lineplot (x=recall_ensemble, y=precision_ensemble, color = 'green')
sns.lineplot (x=recall_rf, y=precision_rf, color = 'red')


plt.title('Precision-recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()